In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np
# import os
# import pandas as pd

2023-06-09 15:54:22.587654: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Dataset preparation

### Set parameters

In [72]:
batch_size = 16
img_height = 184
img_width = 216
img_size = (img_height, img_width)
img_shape = img_size + (3,)
ds_path = "/drive0-storage/Gracia/dataset_1"

### Load dataset

In [74]:
from tensorflow.keras.utils import image_dataset_from_directory

with tf.device("CPU"):
    train_ds = image_dataset_from_directory(ds_path,
                                        validation_split = 0.2,
                                        subset = "training",
                                        seed = 123,
                                        image_size = img_size,
                                        batch_size = batch_size)
    
    val_ds = image_dataset_from_directory(ds_path,
                                      validation_split = 0.2,
                                      subset = "validation",
                                      seed = 123,
                                      image_size = img_size,
                                      batch_size = batch_size)

Found 21408 files belonging to 3 classes.
Using 17127 files for training.
Found 21408 files belonging to 3 classes.
Using 4281 files for validation.


In [66]:
#os.rmdir("/drive0-storage/Gracia/dataset/.ipynb_checkpoints")

In [67]:
class_names = train_ds.class_names
num_classes = len(class_names)

print(class_names)

['bipolar_disorder', 'healthy_controls', 'schizophrenia']


### Test set

In [75]:
with tf.device("CPU"):
    val_batches = tf.data.experimental.cardinality(val_ds)
    test_dataset = val_ds.take(val_batches // 2)
    validation_dataset = val_ds.skip(val_batches // 2)

In [76]:
# Buffered prefetching
AUTOTUNE = tf.data.AUTOTUNE

with tf.device("CPU"):
    train_dataset = train_ds.prefetch(buffer_size = AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size = AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size = AUTOTUNE)

In [51]:
for images in train_ds.take(1):
    img = images[0]
    print(img.shape)  
    print(img.dtype)  
    print(np.min(img),np.max(img))  

(32, 184, 216, 3)
<dtype: 'float32'>
0.0 248.0


In [9]:
len(validation_dataset)

67

## Model building

In [52]:
def build_vgg16():
    model = Sequential()
    
    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=img_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Flatten the output and create fully connected layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))  
    
    return model

In [89]:
with tf.device("GPU"):
    vgg16 = build_vgg16()

    # add rescale layer, chain model
    rescale = tf.keras.layers.Rescaling(1./255)
    inputs = tf.keras.Input(shape = img_shape)
    x = rescale(inputs)
    outputs = vgg16(x)
    model = tf.keras.Model(inputs,outputs) 
    
    # compile model
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate = 1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                  metrics=['accuracy'])

## Model training

In [ ]:
# Callback function
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True
            
callbacks = myCallback()

In [78]:
## batch 16, optimizer adam, lr 1e-5
EPOCH = 8

history_1 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 16)
#             callbacks=[callbacks])

Epoch 1/8
1071/1071 [==============================] - 250s 233ms/step - loss: 0.9988 - accuracy: 0.5578 - val_loss: 1.0100 - val_accuracy: 0.5414
Epoch 2/8
1071/1071 [==============================] - 250s 233ms/step - loss: 0.9951 - accuracy: 0.5583 - val_loss: 1.0066 - val_accuracy: 0.5409
Epoch 3/8
1071/1071 [==============================] - 250s 233ms/step - loss: 0.9916 - accuracy: 0.5583 - val_loss: 1.0013 - val_accuracy: 0.5409
Epoch 4/8
1071/1071 [==============================] - 249s 233ms/step - loss: 0.9707 - accuracy: 0.5629 - val_loss: 0.9381 - val_accuracy: 0.5686
Epoch 5/8
1071/1071 [==============================] - 250s 234ms/step - loss: 0.9094 - accuracy: 0.5923 - val_loss: 0.9004 - val_accuracy: 0.5929
Epoch 6/8
1071/1071 [==============================] - 250s 234ms/step - loss: 0.8673 - accuracy: 0.6165 - val_loss: 0.8661 - val_accuracy: 0.6125
Epoch 7/8
1071/1071 [==============================] - 249s 233ms/step - loss: 0.8294 - accuracy: 0.6319 - val_loss: 0

In [79]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

134/134 [==============================] - 6s 41ms/step - loss: 0.7900 - accuracy: 0.6521


{'loss': 0.7900334000587463, 'accuracy': 0.6520522236824036}

In [12]:
## batch 32, optimizer adam, lr 1e-5
EPOCH = 8

history_2 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 32)
#             callbacks=[callbacks])

Epoch 1/8


2023-06-09 15:54:31.209406: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-09 15:54:32.497949: I tensorflow/stream_executor/cuda/cuda_blas.cc:1633] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


536/536 [==============================] - 165s 300ms/step - loss: 1.0010 - accuracy: 0.5582 - val_loss: 1.0056 - val_accuracy: 0.5433
Epoch 2/8
536/536 [==============================] - 159s 296ms/step - loss: 0.9966 - accuracy: 0.5583 - val_loss: 1.0051 - val_accuracy: 0.5438
Epoch 3/8
536/536 [==============================] - 158s 295ms/step - loss: 0.9939 - accuracy: 0.5583 - val_loss: 1.0018 - val_accuracy: 0.5461
Epoch 4/8
536/536 [==============================] - 159s 296ms/step - loss: 0.9891 - accuracy: 0.5583 - val_loss: 0.9937 - val_accuracy: 0.5414
Epoch 5/8
536/536 [==============================] - 158s 295ms/step - loss: 0.9614 - accuracy: 0.5655 - val_loss: 0.9596 - val_accuracy: 0.5676
Epoch 6/8
536/536 [==============================] - 159s 296ms/step - loss: 0.9051 - accuracy: 0.5959 - val_loss: 0.8993 - val_accuracy: 0.6013
Epoch 7/8
536/536 [==============================] - 158s 295ms/step - loss: 0.8734 - accuracy: 0.6096 - val_loss: 0.9259 - val_accuracy: 0.

In [19]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

67/67 [==============================] - 4s 64ms/step - loss: 0.8444 - accuracy: 0.6236


{'loss': 0.8443752527236938, 'accuracy': 0.6236007213592529}

In [81]:
## batch 16, optimizer adam, lr 1e-4
EPOCH = 8

history_4 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 16)
#             callbacks=[callbacks])

Epoch 1/8
1071/1071 [==============================] - 250s 232ms/step - loss: 0.9995 - accuracy: 0.5581 - val_loss: 1.0102 - val_accuracy: 0.5400
Epoch 2/8
1071/1071 [==============================] - 248s 231ms/step - loss: 0.9953 - accuracy: 0.5583 - val_loss: 1.0071 - val_accuracy: 0.5433
Epoch 3/8
1071/1071 [==============================] - 247s 231ms/step - loss: 0.9945 - accuracy: 0.5583 - val_loss: 1.0102 - val_accuracy: 0.5395
Epoch 4/8
1071/1071 [==============================] - 247s 231ms/step - loss: 0.9940 - accuracy: 0.5583 - val_loss: 1.0080 - val_accuracy: 0.5438
Epoch 5/8
1071/1071 [==============================] - 247s 231ms/step - loss: 0.9940 - accuracy: 0.5583 - val_loss: 1.0083 - val_accuracy: 0.5433
Epoch 6/8
1071/1071 [==============================] - 248s 231ms/step - loss: 0.9935 - accuracy: 0.5583 - val_loss: 1.0097 - val_accuracy: 0.5433
Epoch 7/8
1071/1071 [==============================] - 247s 231ms/step - loss: 0.9934 - accuracy: 0.5583 - val_loss: 1

In [82]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

134/134 [==============================] - 6s 44ms/step - loss: 0.9984 - accuracy: 0.5532


{'loss': 0.9984163641929626, 'accuracy': 0.5531716346740723}

In [27]:
## batch 32, optimizer adam, lr 1e-4
EPOCH = 8

history_5 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 32)
#             callbacks=[callbacks])

Epoch 1/8
536/536 [==============================] - 161s 298ms/step - loss: 1.0030 - accuracy: 0.5583 - val_loss: 1.0070 - val_accuracy: 0.5433
Epoch 2/8
536/536 [==============================] - 159s 297ms/step - loss: 0.9958 - accuracy: 0.5583 - val_loss: 1.0086 - val_accuracy: 0.5423
Epoch 3/8
536/536 [==============================] - 160s 297ms/step - loss: 0.9950 - accuracy: 0.5583 - val_loss: 1.0087 - val_accuracy: 0.5409
Epoch 4/8
536/536 [==============================] - 159s 297ms/step - loss: 0.9942 - accuracy: 0.5583 - val_loss: 1.0129 - val_accuracy: 0.5400
Epoch 5/8
536/536 [==============================] - 159s 297ms/step - loss: 0.9940 - accuracy: 0.5583 - val_loss: 1.0064 - val_accuracy: 0.5470
Epoch 6/8
536/536 [==============================] - 159s 296ms/step - loss: 0.9939 - accuracy: 0.5583 - val_loss: 1.0159 - val_accuracy: 0.5363
Epoch 7/8
536/536 [==============================] - 159s 297ms/step - loss: 0.9940 - accuracy: 0.5583 - val_loss: 1.0084 - val_ac

In [28]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

67/67 [==============================] - 5s 67ms/step - loss: 1.0003 - accuracy: 0.5518


{'loss': 1.0002580881118774, 'accuracy': 0.55177241563797}

In [45]:
## batch 16, optimizer adam, lr 1e-3
EPOCH = 8

history_7 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 16)
#             callbacks=[callbacks])

Epoch 1/8
1071/1071 [==============================] - 99s 92ms/step - loss: 0.9964 - accuracy: 0.5583 - val_loss: 1.0109 - val_accuracy: 0.5414
Epoch 2/8
1071/1071 [==============================] - 98s 92ms/step - loss: 0.9947 - accuracy: 0.5583 - val_loss: 1.0111 - val_accuracy: 0.5409
Epoch 3/8
1071/1071 [==============================] - 98s 92ms/step - loss: 0.9948 - accuracy: 0.5583 - val_loss: 1.0102 - val_accuracy: 0.5409
Epoch 4/8
1071/1071 [==============================] - 98s 92ms/step - loss: 0.9951 - accuracy: 0.5583 - val_loss: 1.0097 - val_accuracy: 0.5423
Epoch 5/8
1071/1071 [==============================] - 98s 92ms/step - loss: 0.9947 - accuracy: 0.5583 - val_loss: 1.0082 - val_accuracy: 0.5428
Epoch 6/8
1071/1071 [==============================] - 98s 92ms/step - loss: 0.9950 - accuracy: 0.5583 - val_loss: 1.0116 - val_accuracy: 0.5400
Epoch 7/8
1071/1071 [==============================] - 99s 92ms/step - loss: 0.9946 - accuracy: 0.5583 - val_loss: 1.0079 - val_ac

In [46]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

134/134 [==============================] - 4s 31ms/step - loss: 1.0002 - accuracy: 0.5527


{'loss': 1.000181794166565, 'accuracy': 0.5527052283287048}

In [54]:
## batch 32, optimizer adam, lr 1e-3
EPOCH = 8

history_8 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 32)
#             callbacks=[callbacks])

Epoch 1/8
536/536 [==============================] - 88s 163ms/step - loss: 1.0086 - accuracy: 0.5562 - val_loss: 1.0076 - val_accuracy: 0.5433
Epoch 2/8
536/536 [==============================] - 87s 163ms/step - loss: 0.9942 - accuracy: 0.5583 - val_loss: 1.0078 - val_accuracy: 0.5438
Epoch 3/8
536/536 [==============================] - 87s 163ms/step - loss: 0.9946 - accuracy: 0.5583 - val_loss: 1.0057 - val_accuracy: 0.5461
Epoch 4/8
536/536 [==============================] - 87s 163ms/step - loss: 0.9942 - accuracy: 0.5583 - val_loss: 1.0106 - val_accuracy: 0.5414
Epoch 5/8
536/536 [==============================] - 87s 163ms/step - loss: 0.9938 - accuracy: 0.5583 - val_loss: 1.0068 - val_accuracy: 0.5452
Epoch 6/8
536/536 [==============================] - 87s 163ms/step - loss: 0.9937 - accuracy: 0.5583 - val_loss: 1.0144 - val_accuracy: 0.5391
Epoch 7/8
536/536 [==============================] - 87s 163ms/step - loss: 0.9941 - accuracy: 0.5583 - val_loss: 1.0082 - val_accuracy:

In [55]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

67/67 [==============================] - 4s 56ms/step - loss: 1.0015 - accuracy: 0.5504


{'loss': 1.0015296936035156, 'accuracy': 0.5503731369972229}

In [84]:
## batch 16, optimizer sgd, lr 1e-5
EPOCH = 8

history_10 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size = 16)
#             callbacks=[callbacks])

Epoch 1/8
1071/1071 [==============================] - 170s 158ms/step - loss: 1.0974 - accuracy: 0.5195 - val_loss: 1.0964 - val_accuracy: 0.5386
Epoch 2/8
1071/1071 [==============================] - 169s 158ms/step - loss: 1.0950 - accuracy: 0.5583 - val_loss: 1.0943 - val_accuracy: 0.5372
Epoch 3/8
1071/1071 [==============================] - 169s 158ms/step - loss: 1.0927 - accuracy: 0.5583 - val_loss: 1.0921 - val_accuracy: 0.5428
Epoch 4/8
1071/1071 [==============================] - 169s 158ms/step - loss: 1.0906 - accuracy: 0.5583 - val_loss: 1.0904 - val_accuracy: 0.5391
Epoch 5/8
1071/1071 [==============================] - 168s 157ms/step - loss: 1.0886 - accuracy: 0.5583 - val_loss: 1.0885 - val_accuracy: 0.5400
Epoch 6/8
1071/1071 [==============================] - 168s 157ms/step - loss: 1.0867 - accuracy: 0.5583 - val_loss: 1.0867 - val_accuracy: 0.5405
Epoch 7/8
1071/1071 [==============================] - 168s 157ms/step - loss: 1.0847 - accuracy: 0.5583 - val_loss: 1

In [85]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

134/134 [==============================] - 5s 40ms/step - loss: 1.0824 - accuracy: 0.5513


{'loss': 1.0824148654937744, 'accuracy': 0.5513059496879578}

In [ ]:
## batch 32, optimizer sgd, lr 1e-5
EPOCH = 8

history_11 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [87]:
## batch 16, optimizer sgd, lr 1e-4
EPOCH = 8

history_13 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH,
            batch_size=16)
#             callbacks=[callbacks])

Epoch 1/8
1071/1071 [==============================] - 93s 86ms/step - loss: 1.0879 - accuracy: 0.5563 - val_loss: 1.0799 - val_accuracy: 0.5409
Epoch 2/8
1071/1071 [==============================] - 92s 86ms/step - loss: 1.0697 - accuracy: 0.5583 - val_loss: 1.0642 - val_accuracy: 0.5433
Epoch 3/8
1071/1071 [==============================] - 92s 86ms/step - loss: 1.0540 - accuracy: 0.5583 - val_loss: 1.0513 - val_accuracy: 0.5414
Epoch 4/8
1071/1071 [==============================] - 92s 86ms/step - loss: 1.0402 - accuracy: 0.5583 - val_loss: 1.0392 - val_accuracy: 0.5433
Epoch 5/8
1071/1071 [==============================] - 92s 86ms/step - loss: 1.0284 - accuracy: 0.5583 - val_loss: 1.0288 - val_accuracy: 0.5447
Epoch 6/8
1071/1071 [==============================] - 92s 86ms/step - loss: 1.0181 - accuracy: 0.5583 - val_loss: 1.0228 - val_accuracy: 0.5405
Epoch 7/8
1071/1071 [==============================] - 92s 86ms/step - loss: 1.0098 - accuracy: 0.5583 - val_loss: 1.0153 - val_ac

In [88]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

134/134 [==============================] - 4s 30ms/step - loss: 1.0066 - accuracy: 0.5494


{'loss': 1.0065914392471313, 'accuracy': 0.549440324306488}

In [ ]:
## batch 32, optimizer sgd, lr 1e-4
EPOCH = 8

history_14 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

In [90]:
## batch 16, optimizer sgd, lr 1e-3
EPOCH = 8

history_16 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

Epoch 1/8
1071/1071 [==============================] - 93s 86ms/step - loss: 1.0305 - accuracy: 0.5578 - val_loss: 1.0077 - val_accuracy: 0.5419
Epoch 2/8
1071/1071 [==============================] - 92s 86ms/step - loss: 0.9930 - accuracy: 0.5583 - val_loss: 1.0102 - val_accuracy: 0.5395
Epoch 3/8
1071/1071 [==============================] - 92s 86ms/step - loss: 0.9927 - accuracy: 0.5583 - val_loss: 1.0075 - val_accuracy: 0.5423
Epoch 4/8
1071/1071 [==============================] - 92s 86ms/step - loss: 0.9928 - accuracy: 0.5583 - val_loss: 1.0112 - val_accuracy: 0.5386
Epoch 5/8
1071/1071 [==============================] - 92s 86ms/step - loss: 0.9929 - accuracy: 0.5583 - val_loss: 1.0062 - val_accuracy: 0.5438
Epoch 6/8
1071/1071 [==============================] - 92s 86ms/step - loss: 0.9926 - accuracy: 0.5583 - val_loss: 1.0080 - val_accuracy: 0.5419
Epoch 7/8
1071/1071 [==============================] - 92s 86ms/step - loss: 0.9926 - accuracy: 0.5583 - val_loss: 1.0119 - val_ac

In [91]:
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

134/134 [==============================] - 4s 30ms/step - loss: 0.9980 - accuracy: 0.5527


{'loss': 0.9979994893074036, 'accuracy': 0.5527052283287048}

In [ ]:
## batch 32, optimizer sgd, lr 1e-3
EPOCH = 8

history_17 = model.fit(
            train_dataset,
            validation_data = validation_dataset,
            epochs = EPOCH)
#             callbacks=[callbacks])

### Training evaluation

In [ ]:
plt.figure(figsize=(8, 8))

# training and validation accuracy
acc = history_1.history['accuracy']
val_acc = history_1.history['val_accuracy']

plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim([min(plt.ylim()),1])
plt.title('Accuracy', size=15, fontweight='bold')

# training and validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch')
plt.ylim([0,1.0])
plt.title('Loss', size=15, fontweight='bold')


plt.subplots_adjust(hspace=0.3)
plt.show()